In [207]:
!pip install category_encoders --quiet

In [208]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
#to ignore warnings
import warnings
warnings.filterwarnings('ignore')
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
import category_encoders as ce
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import OneHotEncoder
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.tree import plot_tree
from sklearn.metrics import accuracy_score, f1_score
import joblib

In [247]:
dff = pd.read_csv(r"/content/Churn (1).csv")

# Load the models

In [248]:
best_gb = joblib.load("/content/best_gb_telecom")
simple_scaler = joblib.load("/content/simple_scaler")
mean_encoder = joblib.load("/content/mean_encoder_model")
enc= joblib.load("/content/ohe_area_code")

# creating test_df   JUST USE "Sample_size:"

In [268]:
sample_size = 10

In [270]:
sample_df = dff.sample(n=sample_size, random_state=42)
sample_df.index = [x for x in range(sample_size)]
df =sample_df.copy()

In [271]:
df                                                      # remove for deployment

,Unnamed: 0,state,area.code,account.length,voice.plan,voice.messages,intl.plan,intl.mins,intl.calls,intl.charge,...,day.calls,day.charge,eve.mins,eve.calls,eve.charge,night.mins,night.calls,night.charge,customer.calls,churn
0,1502,AZ,area_code_510,72,no,0,no,12.7,2,3.43,...,88,46.31,107.9,125,9.17,185.5,81,8.35,0,no
1,2587,MN,area_code_415,53,no,0,no,11.4,2,3.08,...,106,27.9,206,56,17.51,194.7,124,8.76,1,no
2,2654,OK,area_code_408,155,yes,30,no,12.4,5,3.35,...,103,10.47,255.1,110,21.68,225.9,96,10.17,1,no
3,1056,OR,area_code_415,161,no,0,no,8.5,2,2.30,...,109,30.28,146.5,86,12.45,137.6,78,6.19,1,no
4,706,HI,area_code_415,99,no,0,no,8.5,5,2.30,...,81,10.69,231,64,19.64,168.9,121,7.60,1,no
5,107,MI,area_code_510,36,yes,29,no,9.0,6,2.43,...,102,47.84,202.2,76,17.19,187.2,113,8.42,2,no
6,590,WI,area_code_408,117,yes,14,no,8.6,2,2.32,...,81,13.63,219,103,18.62,122.6,102,5.52,1,no
7,2469,WV,area_code_510,152,yes,41,yes,4.3,2,1.16,...,128,24.96,285.6,96,24.28,213.6,80,9.61,1,yes
8,2414,NJ,area_code_510,59,no,0,no,14.8,9,4.00,...,58,33.15,198.5,88,16.87,304.3,110,13.69,0,no
9,1601,MI,area_code_415,115,yes,26,no,4.5,4,1.22,...,110,26.38,230.9,133,19.63,261.6,100,11.77,0,no


#Data Pre-Process

In [272]:
feature_values  = enc.transform(df[['area.code']]).toarray()
feature_labels  = enc.categories_[0]
new_labels  = []
for i in feature_labels:
  new_labels.append("area.code_"+i)
df=pd.concat([pd.DataFrame(feature_values,columns=new_labels),df],axis =1)      # Merged one hot encoding
if df['day.charge'].dtype!=np.number :
  df['day.charge']=df['day.charge'].astype('float')
if df['eve.mins'].dtype!=np.number :
  df['eve.mins']=df['eve.mins'].astype('float')
df=df.dropna()                                               # drop null values
num_cols = df.select_dtypes(include = np.number).columns
cat_cols = df.select_dtypes(include ='object').columns
df["intl.plan"] = df["intl.plan"].map({"no": 0, "yes": 1})   # Binary encoding
df["intl.plan"] = df["intl.plan"].astype("int64")
df["voice.plan"] = df["voice.plan"].map({"no": 0, "yes": 1})
df["voice.plan"] = df["voice.plan"].astype("int64")
df["churn"] = df["churn"].map({"no": 0, "yes": 1})
df["churn"] = df["churn"].astype("int64")
#df = pd.get_dummies(df, columns=['area.code'],dtype = int)   # Label Encoding
df['state_target']= mean_encoder.transform(df['state'])      # State mean encoding
columns_to_drop = ["eve.mins",   "day.mins",  "night.mins",  "intl.mins",'Unnamed: 0','voice.plan','state','churn','area.code']
df = df.drop(columns_to_drop, axis=1)                        # Dropping state varible
feature_to_scaler =['account.length','voice.messages','day.calls','eve.calls','night.calls','intl.calls']
df[feature_to_scaler]= simple_scaler.transform(df[feature_to_scaler])   # Simple scalers

In [273]:
df                          # remove for deployment

,area.code_area_code_408,area.code_area_code_415,area.code_area_code_510,account.length,voice.messages,intl.plan,intl.calls,intl.charge,day.calls,day.charge,eve.calls,eve.charge,night.calls,night.charge,customer.calls,state_target
0,0.0,0.0,1.0,72.0,-3.717873e-17,0,2.0,3.43,88.0,46.31,125.0,9.17,81.0,8.35,0,0.094785
1,0.0,1.0,0.0,53.0,-3.717873e-17,0,2.0,3.08,106.0,27.90,56.0,17.51,124.0,8.76,1,0.171704
2,1.0,0.0,0.0,155.0,3.000000e+01,0,5.0,3.35,103.0,10.47,110.0,21.68,96.0,10.17,1,0.133348
3,0.0,1.0,0.0,161.0,-3.717873e-17,0,2.0,2.30,109.0,30.28,86.0,12.45,78.0,6.19,1,0.163024
4,0.0,1.0,0.0,99.0,-3.717873e-17,0,5.0,2.30,81.0,10.69,64.0,19.64,121.0,7.60,1,0.076366
5,0.0,0.0,1.0,36.0,2.900000e+01,0,6.0,2.43,102.0,47.84,76.0,17.19,113.0,8.42,2,0.162755
6,1.0,0.0,0.0,117.0,1.400000e+01,0,2.0,2.32,81.0,13.63,103.0,18.62,102.0,5.52,1,0.073719
7,0.0,0.0,1.0,152.0,4.100000e+01,1,2.0,1.16,128.0,24.96,96.0,24.28,80.0,9.61,1,0.138211
8,0.0,0.0,1.0,59.0,-3.717873e-17,0,9.0,4.00,58.0,33.15,88.0,16.87,110.0,13.69,0,0.190387
9,0.0,1.0,0.0,115.0,2.600000e+01,0,4.0,1.22,110.0,26.38,133.0,19.63,100.0,11.77,0,0.162755


# Loading best_gb model

In [277]:
feature_names  =  ['account.length', 'voice.messages', 'intl.plan', 'intl.calls',
       'intl.charge', 'day.calls', 'day.charge', 'eve.calls', 'eve.charge',
       'night.calls', 'night.charge', 'customer.calls',
       'area.code_area_code_408', 'area.code_area_code_415',
       'area.code_area_code_510', 'state_target']

In [278]:
df = df[feature_names]                     # Re- ordered features order to work in the model

# Output

In [279]:
best_gb.predict(df)

array([0, 0, 0, 1, 0, 0, 1, 1, 1, 0])